In [ ]:
import yaml
import io
import os
import numpy as np

CWFID_dataset = {
    'annotations': os.path.join(*['CWFID_dataset', 'annotations']),
    'images': os.path.join(*['CWFID_dataset', 'images']),
    'masks': os.path.join(*['CWFID_dataset', 'masks']),
    'SAM_masks': os.path.join(*['CWFID_dataset', 'SAM_masks'])
}


yaml_folder = os.path.join(*['annotations'])
yaml_file_list = os.listdir(yaml_folder)
yaml_file_list = [os.path.join(*[yaml_folder, f]) for f in yaml_file_list if
                  f.endswith('.yaml')
                  and not f.startswith('._')
                  and os.path.isfile(os.path.join(*[yaml_folder, f]))
                  ]
yaml_file_list.sort() # contains ordered list of full paths only of yaml files inside yaml_folder

for filename in yaml_file_list:
    with open(filename, 'r') as stream:
        data_loaded = yaml.safe_load(stream)
        print(f"{data_loaded['filename']}")
        for rec in data_loaded['annotation']:
            coordinates = list(zip(rec['points']['x'], rec['points']['y']))
            n = len(coordinates)
            labels = np.empty(n)
            labels.fill(1)
        pass